In [1]:
import pandas as pd
import glob
import os
import warnings

pd.set_option("display.max_rows", None)
warnings.simplefilter(action='ignore', category=FutureWarning)

folderPath = '../data/in'

In [2]:
excel_files = glob.glob(os.path.join(folderPath, '*.xlsx'))

data = []

for file in excel_files:
    df = pd.read_excel(file, dtype=str)
    year = os.path.basename(file).split('.')[0]
    df["FILE"] = year
    data.append(df)

df = pd.concat(data, ignore_index=True)
df.to_excel('../data/out/output.xlsx', index=False)

In [3]:
# TOTAL_HORAS
try:
    df['TOTAL_HORAS'] = df['TOTAL_HORAS'].str.replace(':', '.')
    df['TOTAL_HORAS'] = df['TOTAL_HORAS'].str.replace('Pendiente', '0')

    df['TOTAL_HORAS'] = pd.to_numeric(df['TOTAL_HORAS'], errors='coerce').fillna(0).round(2)
except:
    print("Proceso terminado")

In [4]:
# NUM CLASE
try:
    df['NUM_CLASE'] = df['NUM_CLASE'].fillna("Sin Definir")
    df['NUM_CLASE'] = df['NUM_CLASE'].str.replace('-1', 'Sin Definir')
except:
    print("Proceso terminado")

In [5]:
# PROGRAMA
try:
    df['PROGRAMA'] = df['PROGRAMA'].fillna("Sin Definir")
except:
    print("Proceso terminado")

In [6]:
# ASIGNATURA
try:
    df['ASIGNATURA'] = df['ASIGNATURA'].fillna("Sin Definir")
except:
    print("Proceso terminado")

In [7]:
replace_values = ["00:00:00", "Pdte", "cancelada", "No ha entregado", "Pendiente", "PENDIENTE", ""]   

# FECHAS
try:
    defaultDate = df['CICLO_LECTIVO'].str.extract(r'(\d{4})')[0] + '-01-01 00:00:00'

    df['FECHA_INICIAL'] = df['FECHA_INICIAL'].fillna(defaultDate)

    df.loc[df['FECHA_INICIAL'].isin(replace_values), 'FECHA_INICIAL'] = defaultDate
    df.loc[df['FECHA_FINAL'].isin(replace_values), 'FECHA_FINAL'] = defaultDate
    
    df['FECHA_INICIAL'] = df['FECHA_INICIAL'].fillna(df['FILE']+'-01-01 00:00:00')
    df['FECHA_FINAL'] = df['FECHA_FINAL'].fillna(df['FECHA_INICIAL'])

    df['FECHA_INICIAL'] = pd.to_datetime(df['FECHA_INICIAL'], errors='coerce')
    df['FECHA_FINAL'] = pd.to_datetime(df['FECHA_FINAL'], errors='coerce')

    df['FECHA_FINAL'] = df['FECHA_FINAL'].fillna(df['FECHA_INICIAL'])
    df['YEAR'] = df['FECHA_INICIAL'].dt.year
except:
    print("Proceso terminado")

In [8]:
# CICLO_LECTIVO
try:
    df['CICLO_LECTIVO'] = df['CICLO_LECTIVO'].fillna(df['YEAR'])
    df['CICLO_LECTIVO'] = df['CICLO_LECTIVO'].fillna(df['FILE'])
except:
    print("Proceso terminado")

In [9]:
# CC
try:
    df['CC_PROFESOR'] = df['CC_PROFESOR'].str.replace(r'[^a-zA-Z0-9]', '', regex=True)
except:
    print("Proceso terminado")

In [10]:
# NOMBRE_PROFESOR
try:
    df['NOMBRE_PROFESOR'] = df['NOMBRE_PROFESOR'].str.upper().str.strip()
except:
    print("Proceso terminado")

In [11]:
# FILL MISSING INFO
try:
    df['CC_PROFESOR'] = df.groupby('NOMBRE_PROFESOR')['CC_PROFESOR'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
    df['NOMBRE_PROFESOR'] = df.groupby('CC_PROFESOR')['NOMBRE_PROFESOR'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
except:
    print("Proceso terminado")

In [12]:
# CC Y NOMBRE
try:
    df['CC_PROFESOR'] = df['CC_PROFESOR'].fillna("0")
    df['NOMBRE_PROFESOR'] = df['NOMBRE_PROFESOR'].fillna("Sin Definir")
except:
    print("Proceso terminado")

In [13]:
# TIPO_CONTRATO
try:
    df['TIPO_CONTRATO'] = df['TIPO_CONTRATO'].fillna("Sin Definir")
except:
    print("Proceso terminado")

In [14]:
df.to_excel('../data/out/clean.xlsx', index=False)